In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [5]:
spark

In [9]:
# rdd = sc.textFile("/labs/laba01/ml-100k/u.data").map(lambda x: x.split("\t"))

In [10]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data")

In [11]:
rdd.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

# 1 task

In [19]:
rdd \
    .filter(lambda x: "\t318\t" in x) \
    .map(lambda x: (x.split("\t")[2], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .sortByKey().collect()

[('1', 4), ('2', 6), ('3', 23), ('4', 79), ('5', 186)]

In [20]:
task1 = rdd \
    .filter(lambda x: "\t318\t" in x) \
    .map(lambda x: (x.split("\t")[2], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .sortByKey().collect()

# 2 task

In [21]:
rdd \
    .map(lambda x: (x.split("\t")[2], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .sortByKey().collect()

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [22]:
task2 = rdd \
    .map(lambda x: (x.split("\t")[2], 1)) \
    .reduceByKey(lambda x, y: x + y) \
    .sortByKey().collect()

In [23]:
import json

In [24]:
d = {'hist_film': [x[1] for x in task1], 'hist_all': [x[1] for x in task2]}

In [42]:
with open('lab01.json', 'w') as f:
    f.write(json.dumps(d))

In [25]:
spark.stop()